In [1]:
import os
import requests
import pandas as pd
from datetime import datetime


# Weather condition code mapping
weather_map = {
    0: "Clear", 1: "Mainly Clear", 2: "Partly Cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing Rime Fog", 51: "Light Drizzle", 53: "Moderate Drizzle",
    55: "Dense Drizzle", 56: "Light Freezing Drizzle", 57: "Dense Freezing Drizzle",
    61: "Slight Rain", 63: "Moderate Rain", 65: "Heavy Rain",
    66: "Light Freezing Rain", 67: "Heavy Freezing Rain",
    71: "Slight Snow", 73: "Moderate Snow", 75: "Heavy Snow", 77: "Snow Grains",
    80: "Slight Rain Showers", 81: "Moderate Rain Showers", 82: "Violent Rain Showers",
    85: "Slight Snow Showers", 86: "Heavy Snow Showers",
    95: "Thunderstorm", 96: "Thunderstorm with Hail", 99: "Thunderstorm with Heavy Hail"
}

def fetch_hourly_weather():
    os.makedirs("data", exist_ok=True)

    # Config
    latitude = 12.9716
    longitude = 77.5946
    start_date = "1990-01-01"
    end_date = datetime.now().strftime('%Y-%m-%d')
    timezone = "auto"

    # Fetch hourly data
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,pressure_msl,precipitation,cloudcover,weathercode",
        "timezone": timezone
    }

    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data["hourly"])
    df["date_time"] = pd.to_datetime(df["time"])
    df["weather_condition"] = df["weathercode"].map(weather_map)

    # Final tidy columns
    final_df = df[[
        "date_time",
        "temperature_2m",
        "relative_humidity_2m",
        "wind_speed_10m",
        "wind_direction_10m",
        "pressure_msl",
        "precipitation",
        "cloudcover",
        "weather_condition"
    ]].rename(columns={
        "temperature_2m": "temperature",
        "relative_humidity_2m": "humidity",
        "wind_speed_10m": "wind_speed",
        "wind_direction_10m": "wind_direction",
        "pressure_msl": "pressure",
        "cloudcover": "cloud_coverage"
    })

    final_df.to_csv("data/hourly_weather.csv", index=False)
    print("✅ Hourly weather data saved to: data/hourly_weather.csv")

fetch_hourly_weather()

✅ Hourly weather data saved to: data/hourly_weather.csv


In [5]:
df = pd.read_csv("data/hourly_weather.csv")
df.head()

,date_time,temperature,humidity,wind_speed,wind_direction,pressure,precipitation,cloud_coverage,weather_condition
0,1990-01-01 00:00:00,19.8,89.0,11.0,109.0,1014.8,0.0,88.0,Overcast
1,1990-01-01 01:00:00,19.7,89.0,10.4,110.0,1014.5,0.0,75.0,Partly Cloudy
2,1990-01-01 02:00:00,19.3,91.0,8.0,108.0,1013.7,0.0,71.0,Partly Cloudy
3,1990-01-01 03:00:00,19.1,92.0,6.0,107.0,1013.5,0.0,34.0,Mainly Clear
4,1990-01-01 04:00:00,18.6,95.0,4.9,107.0,1013.4,0.0,40.0,Mainly Clear
